# Reading VIIRS Tile Data

Author: James Yoon (jyyoon@uw.edu)

---

Last updated: 26 May 2025

This script reads in VIIRS tiles, which are archived with sinusoidal geographic coordinates. This script automagically takes in the data, identifies which tile it is, and converts it to latitude and longitude. To ease data storage, it regrids the VIIRS data into a regular grid for saving into a NetCDF.

In [33]:
# Common imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob
from datetime import timedelta

# More uncommon imports -- make sure you have satellite_processing_functions.py in the same directory
import netCDF4 as nc
import geopandas as gpd
import shapely
import xesmf as xe
import re # Import regular expressions

from satellite_processing_functions import convert_to_string, new_grid, read_scattered_TROPOMI_data, regrid_scattered_data, read_viirs_data
from processing_viirs_sinusoidal_grid import sinusoidal_grid;

# Mapping functions -- not required to process the data, but nice to plot as a sanity check!
import cartopy.crs as ccrs
import cartopy.feature as cfeature

%config InlineBackend.figure_format='retina'
plt.rcParams["font.family"] = "Arial"

def basemap():
    # Creates a fig and ax object with a map of the globe using the Cartopy library
    fig = plt.figure(figsize=(10,5));
    ax = fig.add_subplot(111, projection=ccrs.PlateCarree()); # Can change the projection using different ccrs.__()
    color = 'gray' # Color of the map lines

    # Add in the coastlines, states, and country borders with 50 m resolution.
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.5, edgecolor = color);
    ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.5, edgecolor = color);
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=0.5, edgecolor = color);

    return fig, ax;

# import warnings
# warnings.filterwarnings("ignore")

---
# Parameters

Setting up some parameters, such as the time period we're interested in and the geographic region!

In [24]:
# Accounts for the 8-day (16-day period) VIIRS dataset
data_type = 'VIIRS_downscaling'

year_start = 2018
doys_start = 121

year_end = 2021
doys_end = 365

starting_date = pd.to_datetime(f'{year_start}-{doys_start}', format='%Y-%j')
ending_date = pd.to_datetime(f'{year_end}-{doys_end}', format='%Y-%j')
time_array = pd.to_datetime(np.arange(starting_date, ending_date, timedelta(days = 1)))

In [26]:
# Geolocations
region = 'PNW'

if region == 'California':
    # geolocation_bounds = [-124.541016, 32.405745, -113.686523, 42.049293]
    geolocation_bounds = [-123.206177,37.028869,-121.393433,38.436380]
elif region == 'PNW':
    geolocation_bounds = [-129.287109,43.197167,-115.378418,51.096623]
elif region == 'Sudd':
    geolocation_bounds = [29.245605, 5, 32.515283, 9.774025]

cell_size_lat = 0.025
cell_size_lon = 0.025

new_grid_interp = new_grid(geolocation_bounds[0], geolocation_bounds[1], geolocation_bounds[2], geolocation_bounds[3], cell_size_lat, cell_size_lon) # Creates new grid to interpolate onto.

---
# Part II: Data Processing

We'll use the read_viirs_function from the satellite_processing_functions script to get the raw data, followed by regridding onto a regular grid.

In [27]:
def compile_viirs_data(viirs_fn, geolocation_bounds):
    df_day = pd.DataFrame()

    # Loop through files in the glob array
    for viirs_f in viirs_fn: 
        tile_data = read_viirs_data(viirs_f, geolocation_bounds)

        if len(tile_data) > 0:
            df_day = pd.concat([df_day, tile_data], ignore_index=True)

    return df_day;

In [ ]:
viirs_data = pd.DataFrame() # Will contain the final VIIRS product

for time in time_array: # times on VIIRS grid
    viirs_data = pd.DataFrame()
    year = time.year
    doy = str(time.dayofyear).zfill(3)
    
    viirs_fn = glob(f"/home/disk/hermes/jyyoon/Vegetation_Indices/VIIRS/{region}/VNP13A1.A{year}{doy}*.h5")

    if len(viirs_fn) == 0:
        print(f"No files available for {time}")
    else:
        df_tiles = compile_viirs_data(viirs_fn, geolocation_bounds);
        start_time = df_tiles['start_time'].values[0];
        end_time = df_tiles['end_time'].values[0];
    
        # Regrid the VIIRS/MODIS sinusoidal grid to a regular grid!
        gdf = gpd.GeoDataFrame(df_tiles, geometry = gpd.points_from_xy(df_tiles['lon'], df_tiles['lat'])) # Create a GeoDataFrame
        gdf = regrid_scattered_data(gdf, new_grid_interp, data_type);
        gdf['start_time'] = start_time;
        gdf['end_time'] = end_time;
        gdf = gdf.set_index(['start_time', 'lat', 'lon']).to_xarray()

        if len(viirs_data) == 0:
            viirs_data = gdf;
        else:
            viirs_data = xr.concat([viirs_data, gdf], dim = 'start_time')

    if len(viirs_data) > 0:
        viirs_data.to_netcdf(f"PNW_VIIRS/PNW_VIIRS_{year}-{doy}.nc")
    print(f'Finished processing {time} ****************************')

Skipped this tile because data doesn't intersect bounds.
Finished processing 2018-05-01 00:00:00 ****************************
No files available for 2018-05-02 00:00:00
Finished processing 2018-05-02 00:00:00 ****************************
No files available for 2018-05-03 00:00:00
Finished processing 2018-05-03 00:00:00 ****************************
No files available for 2018-05-04 00:00:00
Finished processing 2018-05-04 00:00:00 ****************************
No files available for 2018-05-05 00:00:00
Finished processing 2018-05-05 00:00:00 ****************************
No files available for 2018-05-06 00:00:00
Finished processing 2018-05-06 00:00:00 ****************************
No files available for 2018-05-07 00:00:00
Finished processing 2018-05-07 00:00:00 ****************************
No files available for 2018-05-08 00:00:00
Finished processing 2018-05-08 00:00:00 ****************************
Skipped this tile because data doesn't intersect bounds.
Finished processing 2018-05-09 00